In [ ]:
import numpy as np
import pandas as pd
from heapq import heapify, heappush, heappop
from sklearn.decomposition import TruncatedSVD as tsvd

In [ ]:
config = pd.read_csv('config.csv').iloc[0]

user = int(config['id'])
k_value = int(config['k'])

In [ ]:
data_df = pd.read_csv(config['dataset_file'], delimiter=' ')
data_df.columns = ['user_ids', 'image_ids', 'timpstamp']

In [ ]:
all_images = set()
user_likes = dict()

In [ ]:
usr = []
imgs = []
vals = []
for row in data_df.values:
    # usr.append(user_ids_index[row[0]])
    # imgs.append(image_ids_index[row[1]])
    # vals.append(1)
    if row[0] not in user_likes:
        user_likes[row[0]] = set()
    user_likes[row[0]].add(row[1])
    all_images.add(row[1])

In [ ]:
all_users = list(user_likes.keys())

In [ ]:
# user_ids = data_df['user_ids'].unique()
user_ids_index = dict()
# image_ids = data_df['image_ids'].unique()
image_ids_index = dict()

In [ ]:
i = 0
for user_id in user_likes.keys():
    user_ids_index[user_id] = i
    i += 1

i = 0
for img_id in all_images:
    image_ids_index[img_id] = i
    i += 1

In [ ]:
for row in data_df.values:
    usr.append(user_ids_index[row[0]])
    imgs.append(image_ids_index[row[1]])
    vals.append(1)


In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
sparse_matrix = csr_matrix((vals, (usr, imgs)))

In [ ]:
sparse_matrix.shape

(63497, 178787)

no of components value = 1000, value found after experimentation with different values

random_state = 17, any value to get the same result

In [ ]:
svd = tsvd(n_components=1000, random_state=17)
matrix = svd.fit_transform(sparse_matrix)

In [ ]:
from numpy.linalg import norm

def cosine_distance(a, b):
    return np.dot(a, b)/(norm(a)*norm(b))

In [ ]:
def neighbors(user, k_value):
    global user_likes
    user_img_id_set = user_likes[user]
    jcc_heap = []
    corr_heap = []
    heapify(jcc_heap)
    heapify(corr_heap)
    
    for some_user,img_id_set in user_likes.items():
        if some_user == user:
            continue
        #calculating jaccard index
        jcc_ind = len(img_id_set.intersection(user_img_id_set))/len(img_id_set.union(user_img_id_set))
        heappush(jcc_heap, (jcc_ind, some_user))
        # no_of_common_img = len(img_id_set.intersection(user_img_id_set))
        # heappush(jcc_heap, (no_of_common_img, some_user))
        if(len(jcc_heap) > k_value):
            heappop(jcc_heap)
    
    user_idx = user_ids_index[user]
    user_matrix = matrix[user_idx]
    for i in range(len(matrix)):
        if(i == user_idx):
            continue
        heappush(corr_heap, (cosine_distance(user_matrix, matrix[i]), i))
        if(len(corr_heap) > k_value+3):
            heappop(corr_heap)
    jcc_users = [user_ for val, user_ in sorted(jcc_heap, reverse=True)]
    svd_min_users = [all_users[i] for val, i in corr_heap]
    print(jcc_users)
    print('No. of common users between jaccard method and correlation vector method')
    print(set(jcc_users).intersection(set(svd_min_users)))

    return [all_users[t[1]] for t in sorted(corr_heap, key=lambda x: -x[0])]

In [ ]:
outFile = open(config['output_file'], 'w')

sol_item_set = set()
no_users = 0
for user_id in neighbors(user, k_value):
    new_user = True
    for item_id in user_likes[user_id]:
        if item_id not in user_likes[user] and item_id not in sol_item_set:
            if new_user:
                no_users += 1
                new_user = False
            if no_users > k_value:
                break
            sol_item_set.add(item_id)
            outFile.write(str(item_id) + ' ' + str(user_id) + '\n')
print(no_users)
outFile.close()

[1442856, 1762419, 2840102, 3678210, 3621636]
No. of common users between jaccard method and correlation vector method
set()
5
